In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv('event_data.csv')

# Display the first few rows to understand the structure
df.head()


,user_id,event_date,event_type,purchase_amount
0,c40e6a,2019-07-29 00:02:15,registration,NaN
1,a2b682,2019-07-29 00:04:46,registration,NaN
2,9ac888,2019-07-29 00:13:22,registration,NaN
3,93ff22,2019-07-29 00:16:47,registration,NaN
4,65ef85,2019-07-29 00:19:23,registration,NaN


In [3]:
# Convert event_date to datetime
df['event_date'] = pd.to_datetime(df['event_date'])

# Extract week number from event_date as cohort_id
df['cohort_id'] = df['event_date'].dt.isocalendar().week

# Display updated dataframe with cohort_id
df.head()


,user_id,event_date,event_type,purchase_amount,cohort_id
0,c40e6a,2019-07-29 00:02:15,registration,NaN,31
1,a2b682,2019-07-29 00:04:46,registration,NaN,31
2,9ac888,2019-07-29 00:13:22,registration,NaN,31
3,93ff22,2019-07-29 00:16:47,registration,NaN,31
4,65ef85,2019-07-29 00:19:23,registration,NaN,31


In [4]:
# Count unique users in cohort ID 33
cohort_33_users = df[df['cohort_id'] == 33]['user_id'].nunique()
print(f"Number of unique users in cohort ID 33: {cohort_33_users}")


Number of unique users in cohort ID 33: 5102


In [5]:
# Calculate registration date for each user
df['registration_date'] = df.groupby('user_id')['event_date'].transform('min')

# Calculate lifetime in weeks
df['lifetime_week'] = ((df['event_date'] - df['registration_date']).dt.days / 7).astype(int)

# Display updated dataframe with lifetime_week
df.head()


,user_id,event_date,event_type,purchase_amount,cohort_id,registration_date,lifetime_week
0,c40e6a,2019-07-29 00:02:15,registration,NaN,31,2019-07-29 00:02:15,0
1,a2b682,2019-07-29 00:04:46,registration,NaN,31,2019-07-29 00:04:46,0
2,9ac888,2019-07-29 00:13:22,registration,NaN,31,2019-07-29 00:13:22,0
3,93ff22,2019-07-29 00:16:47,registration,NaN,31,2019-07-29 00:16:47,0
4,65ef85,2019-07-29 00:19:23,registration,NaN,31,2019-07-29 00:19:23,0


In [6]:
# Assuming df is already loaded and processed up to Step 4

# Group by cohort_id and user_id to identify unique users in each cohort
cohorts = df.groupby(['cohort_id', 'user_id']).agg({'event_date': 'min'}).reset_index()

# Merge back to calculate lifetime_week correctly
cohorts = cohorts.merge(df[['user_id', 'event_date', 'lifetime_week']], on='user_id')

# Count the number of unique users in each cohort
cohort_sizes = cohorts.groupby('cohort_id').agg({'user_id': 'nunique'}).reset_index()
cohort_sizes.columns = ['cohort_id', 'total_users']

# Calculate retention for each cohort and lifetime_week
retention = cohorts.groupby(['cohort_id', 'lifetime_week']).agg({'user_id': 'nunique'}).reset_index()
retention = retention.merge(cohort_sizes, on='cohort_id')
retention['retention_rate'] = retention['user_id'] / retention['total_users']

# Pivot the retention table to get a summary table
retention_summary = retention.pivot(index='cohort_id', columns='lifetime_week', values='retention_rate')

# Display the retention rate summary table
retention_summary



lifetime_week,0,1,2,3,4
cohort_id,,,,,
31,1.0,0.804051,0.475949,0.244557,0.061772
32,1.0,0.837474,0.498943,0.239429,0.032241
33,1.0,0.881223,0.546648,0.177577,0.023912
34,1.0,0.855674,0.437779,0.155297,0.020912
35,1.0,0.593780,0.305066,0.115422,0.019558


In [9]:
# Assuming retention_summary table is already calculated and available

# Check if retention_summary is already calculated, if not, calculate it
try:
    retention_summary
except NameError:
    print("Please calculate retention_summary first.")  # Handle if retention_summary is not defined

# Calculate 3-week retention rate for cohort ID 32
cohort_id = 32
week = 3

# Check if cohort_id exists in retention_summary
if cohort_id in retention_summary.index:
    retention_rate_3w = retention_summary.loc[cohort_id, week]
    print(f"3-week retention rate for cohort ID {cohort_id}: {retention_rate_3w:.2%}")
else:
    print(f"Cohort ID {cohort_id} not found in retention_summary.")



3-week retention rate for cohort ID 32: 23.94%


In [10]:
# Assuming df is already loaded and processed up to Step 4

# Filter for paying users
paying_users = df[df['event_type'] == 'purchase']

# Group by cohort_id and user_id to identify unique paying users in each cohort
paying_cohorts = paying_users.groupby(['cohort_id', 'user_id']).agg({'purchase_amount': 'sum'}).reset_index()

# Merge back to calculate lifetime_week correctly
paying_cohorts = paying_cohorts.merge(df[['user_id', 'event_date', 'lifetime_week']], on='user_id')

# Calculate ARPPU for each cohort and lifetime_week
arppu = paying_cohorts.groupby(['cohort_id', 'lifetime_week']).agg({'purchase_amount': 'mean'}).reset_index()

# Pivot the ARPPU table to get a summary table
arppu_summary = arppu.pivot(index='cohort_id', columns='lifetime_week', values='purchase_amount')

# Display the ARPPU summary table
arppu_summary


lifetime_week,0,1,2,3,4
cohort_id,,,,,
31,30.290955,31.024735,30.881671,33.103448,28.780488
32,34.916597,34.787823,34.565343,36.974220,36.625000
33,34.687403,35.490483,34.606782,35.686720,31.511628
34,35.014777,35.585452,35.494628,33.655172,34.583333
35,34.968529,34.971228,34.026882,33.447684,33.521127


In [11]:
# Assuming arppu_summary table is already calculated and available

# Check if arppu_summary is already calculated, if not, calculate it
try:
    arppu_summary
except NameError:
    print("Please calculate arppu_summary first.")  # Handle if arppu_summary is not defined

# Calculate 3-week ARPPU for cohort ID 31
cohort_id = 31
week = 3

# Check if cohort_id exists in arppu_summary
if cohort_id in arppu_summary.index:
    arppu_3w = arppu_summary.loc[cohort_id, week]
    print(f"3-week ARPPU for cohort ID {cohort_id}: ${arppu_3w:.2f}")
else:
    print(f"Cohort ID {cohort_id} not found in arppu_summary.")


3-week ARPPU for cohort ID 31: $33.10


In [12]:
import pandas as pd

# Assuming df is already loaded and processed up to Step 4

# Filter for purchase events only
purchase_events = df[df['event_type'] == 'purchase']

# Group by user_id to find the earliest purchase date
first_purchase_dates = purchase_events.groupby('user_id')['event_date'].min().reset_index()

# Merge with the original dataset to get registration date for each user
user_registration_dates = df[df['event_type'] == 'registration'][['user_id', 'event_date']].rename(columns={'event_date': 'registration_date'})

# Merge to calculate time to first purchase
user_first_purchase = pd.merge(user_registration_dates, first_purchase_dates, on='user_id', how='left')
user_first_purchase['time_to_first_purchase'] = (user_first_purchase['event_date'] - user_first_purchase['registration_date']).dt.total_seconds()

# Calculate median time to first purchase
median_time_to_first_purchase = user_first_purchase['time_to_first_purchase'].median()

# Display or print the median time in seconds
print(f"Median time between registration and first purchase: {int(median_time_to_first_purchase)} seconds")


Median time between registration and first purchase: 434774 seconds
